In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
from statsmodels.stats import weightstats 

pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок
df = pd.read_csv('stud_math.xls',index_col = False)


Посмотрим на переменные, которые содержит датасет:

1 school — аббревиатура школы, в которой учится ученик

2 sex — пол ученика ('F' - женский, 'M' - мужской)

3 age — возраст ученика (от 15 до 22)

4 address — тип адреса ученика ('U' - городской, 'R' - за городом)

5 famsize — размер семьи('LE3' <= 3, 'GT3' >3)

6 Pstatus — статус совместного жилья родителей ('T' - живут вместе 'A' - раздельно)

7 Medu — образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

8 Fedu — образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

9 Mjob — работа матери ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)

10 Fjob — работа отца ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)

11 reason — причина выбора школы ('home' - близость к дому, 'reputation' - репутация школы, 'course' - образовательная программа, 'other' - другое)

12 guardian — опекун ('mother' - мать, 'father' - отец, 'other' - другое)

13 traveltime — время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.)

14 studytime — время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов)

15 failures — количество внеучебных неудач (n, если 1<=n<=3, иначе 0)

16 schoolsup — дополнительная образовательная поддержка (yes или no)

17 famsup — семейная образовательная поддержка (yes или no)

18 paid — дополнительные платные занятия по математике (yes или no)

19 activities — дополнительные внеучебные занятия (yes или no)

20 nursery — посещал детский сад (yes или no)

21 higher — хочет получить высшее образование (yes или no)

22 internet — наличие интернета дома (yes или no)

23 romantic — в романтических отношениях (yes или no)

24 famrel — семейные отношения (от 1 - очень плохо до 5 - очень хорошо)

25 freetime — свободное время после школы (от 1 - очень мало до 5 - очень мого)

26 goout — проведение времени с друзьями (от 1 - очень мало до 5 - очень много)

27 health — текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо)

28 absences — количество пропущенных занятий

29 score — баллы по госэкзамену по математике

Рекомендации по выполнению проекта
Проведите первичную обработку данных. Так как данных много, стоит написать функции, которые можно применять к столбцам определённого типа.
Посмотрите на распределение признака для числовых переменных, устраните выбросы.
Оцените количество уникальных значений для номинативных переменных.
По необходимости преобразуйте данные
Проведите корреляционный анализ количественных переменных
Отберите не коррелирующие переменные.
Проанализируйте номинативные переменные и устраните те, которые не влияют на предсказываемую величину (в нашем случае — на переменную score).
Не забудьте сформулировать выводы относительно качества данных и тех переменных, которые вы будете использовать в дальнейшем построении модели.

Вас пригласили поучаствовать в одном из проектов UNICEF — международного подразделения ООН, чья миссия состоит в повышении уровня благополучия детей по всему миру. 

Суть проекта — отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

И сделать это можно с помощью модели, которая предсказывала бы результаты госэкзамена по математике для каждого ученика школы (вот она, сила ML!). Чтобы определиться с параметрами будущей модели, проведите разведывательный анализ данных и составьте отчёт по его результатам. 

In [2]:
# числовые значения в единый формат
df = df.astype({'age': 'float64'})

In [3]:
#все названия колонок с маленькой буквы
df.columns = [x.lower() for x in list(df.columns)]


In [4]:
# переименовываем колонку чтобы из одного слова без разделителей
df.rename({'studytime, granular': 'studytime2'}, axis=1, inplace=True)

In [5]:
# удаляем данные с результатом теста = 0, как не релевантныею. "0" = тест не сдавался
df = df[df.score != 0]

In [6]:
# выбираем колонки по типу данных и делим базу на две по этому признаку. Позднее поменялся подход.
# Сначала рассматривается вся база, а стиранее строк будет после отбора колонок.
Ncols = df.select_dtypes(include=['number']).columns
NumDF = df.select_dtypes(include=['number'])
Ncols2 = Ncols.drop('score')
Ocols = df.columns.drop(Ncols2)
ObjDF = df.drop(Ncols2, axis=1)

In [7]:
# Зполняем НАН
for column in df.columns:
    if column in Ncols2:
        median_ = df[column].median()
        df[column].fillna(value=median_, inplace=True, axis=0)
    else:
        mode_ = df[column].mode()[0]
        df[column].fillna(value=mode_, inplace=True, axis=0)

In [8]:
# найдены и убраны выбросы.
# создаем функцию которая создает таблицу со статистикой по каждой колонке

def calc_iqr(columns):
    out = []
    for column in list(columns):
        IQR = df[column].quantile(0.75) - df[column].quantile(0.25)
        perc25 = df[column].quantile(0.25)
        perc75 = df[column].quantile(0.75)
        Lbound = ((perc25 - 1.5 * IQR))
        Ubound = ((perc75 + 1.5 * IQR))
        out.append((column, IQR, perc25, perc75, Lbound,Ubound))
    temp = pd.DataFrame(out, columns=('columns', 'IQR', 'perc25','perc75','Lbound','Ubound'))
    return temp

# Создаем таблицу с результатами вычислений функции

x = calc_iqr(Ncols)

# Цикл удаляет из числовой ДФ строки в которых функция обнаружилы выбросы

df2= df.copy()
for y in range(len(x)):
    lbound = x.iloc[y,4]
    ubound = x.iloc[y,5]
    name = x.iloc[y,0]
    if name != 'failures':  # потому что все считается выбросами, хотя информация важная
         df2 = df2[(df2[name] >= lbound) & (df2[name] <= ubound)]

In [9]:
# Для количественных переменных построены гистограммы распределений и сделаны выводы.
# нужно заменять название обрабатываемой DF внутри функции
def get_boxplot(column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y='score',
                data=df2.loc[df2.loc[:, column].isin(df2.loc[:, column].value_counts().index[:10])],
               ax=ax)
    plt.xticks(rotation=0)
    ax.set_title('Boxplot for ' + column)
    plt.show()


In [10]:
 # выводы по Номинативным
# school - щкола GP немного лучше в среднем но и разброс больше
# Sex - у женщин максимальный балл выше, средний выше на 7-10 баллов, разброс немного меньше
# address - городские жители слегка выше в среднем, но разброс больше
# famsize -размер семьи. В большой семье слегка ниже в среднем и разброс больше
# pstatus - кто живет раздельно в среднем так же кто и вместе
# mjob - health лучше всего средняя, минимальная, максимальная
# fjob - только те у кого отец учитель - оценки заметно выше
# reason - в среднем одинаково
# guardian - в среднем одинаково
# schoolsup - БЕЗ доп образования сдали тест ЛУЧШЕ в среднем, максимальном, разброс выше
# famsup - одинаково
# paid - дополнительные  занятия - не влияют
# activities - ВНЕучебные не влияют
# nursery - детский сад не влияет
# higher - те кто хочет поступать в высшее стараются получить лучший балл, и это им удается. средняя выше, основная масса
# наблюдений в более высоком диапазоне, выше максимальные значения
# internet - не влияет
# romantic - не влияет

In [22]:

Final_list = []

# Дополнительный анализ номинативных переменных Т-тес
def get_stat_dif(column):
    cols = df2.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df2.loc[df2.loc[:, column] == comb[0], 'score'],
                        df2.loc[df2.loc[:, column] == comb[1], 'score']).pvalue \
            <= 0.05/len(combinations_all):
            if column != 'score':
                Final_list.append(column)
            print('Найдены статистически значимые различия для колонки', column)
            break

for col in Ocols:
     get_stat_dif(col)
        
        
min_corr = 0.15

corr = df2.corr()['score']
corr =corr[abs(corr.values) > min_corr]
add = list(map(lambda x: x,corr.keys()))
for x in add:
    if x != 'score':
        Final_list.append(x)

Final_list

Найдены статистически значимые различия для колонки school
Найдены статистически значимые различия для колонки sex
Найдены статистически значимые различия для колонки address
Найдены статистически значимые различия для колонки schoolsup
Найдены статистически значимые различия для колонки score


['school',
 'sex',
 'address',
 'schoolsup',
 'age',
 'medu',
 'fedu',
 'failures',
 'goout',
 'absences']

In [12]:
#выводы
# В листе Final_list -наименования колонок которые можно оставть для анлиза данных
# для изменения порога уровня ПРИЕМЛИМОЙ корреляции помжно поменять, поменяв значение переменной  min_corr
#